## Drugs Arrested Case

### ปัจจัยที่ส่งผลต่อจำนวนคดีผู้ต้องหายาเสพติด
#### by กลุ่ม ทุกปัญหามีทางเข้าเสมอ
---

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import widgets, interact
import re

เกริ่นเรื่องปัญหายาเสพติดในไทย

นำเสนอข้อมูลหลักที่มาใช้ใน งานครั้งนี้
- ดึงข้อมูล จำนวนคดี/ผู้ต้องหายาเสพติดในไทยมา โดยอธิบายว่าเรานำข้อมูลตัวนี้มาเป็นเลขตัวแทนของปัญหาการเสพ อาจจะไม่ได้เป็นตัวแทยที่สามารถแทยได้ร้อยเปอร์เซ็นเพียงแค่ดูพวกแนวโน้ม การกระจุกตัวได้เฉยๆ เพราะมันยังมีปัจจัยอื่นๆอีกมากคือ บางพื้นที่ บางช่วงเวลาอาจไม่ได้มีความรัดกุมในการดำเนินคดีไรงี้

- ข้อมูลที่เป็นตัวชี้วัดเศรษฐกิจ  ไม่แน่ใจควรทำมั้ยนะ แบบเห็นมีกราฟจำนวนคดีรวมทั้งหมดของแต่ละปีแล้วเหมือนช่วงโควิดจะเยอะกว่าช่วงอื่นนิดหน่อยงี้เผื่อเกี่ยว

- จำนวนประชากรแต่ละจังหวัด ทำเป็นความหนาแน่นได้มั้ยนะ??

- จำนวนคดีเคส เรียงตามจังหวัด เอาปีล่าสุด - เลือกจังหวัดแสดงผลได้
- ดูเทรนด์ rank top 3 แต่ละปี
- เอาจังหวัดที่น่าสนใจ(สถิติอยู่ในท็อป3บ่อย) ex.นครศรีธรรมราช กรุงเทพ ปทุม สงขลา ขอนแก่น
- ประชากรหนาแน่น ex.กรุงเทพ ขอนแก่น
- ดูเรื่องนักท่องเที่ยว ฟีลเมืองเศรษฐกิจ ex.ชลบุรี
- หัวข้อภาคใต้ เน้นนครศรี + เรื่องกระท่อม + อาชญากรรมอื่น
- การกินเหล้า & เล่นยาสวนทางกัน - 